In [1]:
from openmc_model import baby_model, cllif_nat

my_model = baby_model()

isotopes ['Al27', 'Ar36', 'Ar38', 'Ar40', 'B10', 'B11', 'Be9', 'C12', 'C13', 'Ca40', 'Ca42', 'Ca43', 'Ca44', 'Ca46', 'Ca48', 'Cl35', 'Cl37', 'Co59', 'Cr50', 'Cr52', 'Cr53', 'Cr54', 'Cu63', 'Cu65', 'F19', 'Fe54', 'Fe56', 'Fe57', 'Fe58', 'H1', 'H2', 'He3', 'He4', 'K39', 'K40', 'K41', 'Li6', 'Li7', 'Mg24', 'Mg25', 'Mg26', 'Mn55', 'Mo100', 'Mo92', 'Mo94', 'Mo95', 'Mo96', 'Mo97', 'Mo98', 'N14', 'N15', 'Na23', 'Nb93', 'Ni58', 'Ni60', 'Ni61', 'Ni62', 'Ni64', 'O16', 'O17', 'O18', 'P31', 'Pb204', 'Pb206', 'Pb207', 'Pb208', 'S32', 'S33', 'S34', 'S36', 'Si28', 'Si29', 'Si30', 'Ti46', 'Ti47', 'Ti48', 'Ti49', 'Ti50', 'W182', 'W183', 'W184', 'W186', 'Zr90', 'Zr91', 'Zr92', 'Zr94', 'Zr96']
Searching libraries with the following priority {'ENDFB-8.0-NNDC': 1}
Isotopes found matching library requirements 556
Isotopes found matching particle requirements 1789
Isotopes found matching isotope requirements 345
Isotopes found matching all requirements 87
             library remote_file  \
1233  ENDFB-8.0-N

In [2]:

for cell_id, cell in my_model.geometry.get_all_cells().items():
    if cell.fill == cllif_nat:
        salt_cell = cell
    
    cell.fill = None

In [3]:
salt_surfaces = list(salt_cell.region.get_surfaces().values())

salt_surfaces

[Surface
 	ID                 =	7
 	Name               =	
 	Type               =	z-plane
 	Boundary           =	transmission
 	Coefficients       
 z0                  =	107.401,
 Surface
 	ID                 =	8
 	Name               =	
 	Type               =	z-plane
 	Boundary           =	transmission
 	Coefficients       
 z0                  =	113.91922,
 Surface
 	ID                 =	16
 	Name               =	
 	Type               =	z-cylinder
 	Boundary           =	transmission
 	Coefficients       
 x0                  =	587
 y0                  =	60
 r                   =	7.0,
 Surface
 	ID                 =	22
 	Name               =	
 	Type               =	z-cylinder
 	Boundary           =	transmission
 	Coefficients       
 x0                  =	587
 y0                  =	60
 r                   =	0.439,
 Surface
 	ID                 =	23
 	Name               =	
 	Type               =	z-plane
 	Boundary           =	transmission
 	Coefficients       
 z0                  =	108

In [4]:
import openmc

surf_filter = openmc.SurfaceFilter(salt_surfaces)

from_cell_filter = openmc.CellFromFilter([cell for cell in my_model.geometry.get_all_cells().values()if cell != salt_cell]) 
# from_cell_filter = openmc.CellFromFilter([salt_cell]) 
# from_cell_filter = openmc.CellFromFilter([cell for cell in my_model.geometry.get_all_cells().values()]) 


In [5]:
tally = openmc.Tally(name="salt tally")
tally.filters = [surf_filter, from_cell_filter]
tally.scores = ["current"]

my_model.tallies = [tally]

In [6]:
my_model.materials = []

In [7]:
my_model.run()

                                %%%%%%%%%%%%%%%
                           %%%%%%%%%%%%%%%%%%%%%%%%
                        %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                      %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                    %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                   %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                                    %%%%%%%%%%%%%%%%%%%%%%%%
                                     %%%%%%%%%%%%%%%%%%%%%%%%
                 ###############      %%%%%%%%%%%%%%%%%%%%%%%%
                ##################     %%%%%%%%%%%%%%%%%%%%%%%
                ###################     %%%%%%%%%%%%%%%%%%%%%%%
                ####################     %%%%%%%%%%%%%%%%%%%%%%
                #####################     %%%%%%%%%%%%%%%%%%%%%
                ######################     %%%%%%%%%%%%%%%%%%%%
                #######################     %%%%%%%%%%%%%%%%%%
                 #######################     %%%%%%%%%%%%%%%%%
                 #####################

PosixPath('/home/remidm/baby-1L-run-4/analysis/neutron/statepoint.100.h5')

In [8]:
salt_cell

Cell
	ID             =	7
	Name           =	
	Fill           =	None
	Region         =	(7 -8 -16 (22 | -23 | 24))
	Rotation       =	None
	Translation    =	None
	Volume         =	None

In [9]:
sp = openmc.StatePoint(f"statepoint.{my_model.settings.batches}.h5")
tally = sp.get_tally(name="salt tally").get_pandas_dataframe()

# filter where mean is not zero
tally = tally[tally['mean'] != 0]

tally

,surface,cellfrom,nuclide,score,mean,std. dev.
25,7,9,total,current,0.065932,0.000239
29,7,14,total,current,0.040766,0.000201
30,7,13,total,current,0.220096,0.000384
65,8,14,total,current,0.024657,0.000155
66,8,13,total,current,0.228077,0.000356
97,16,9,total,current,0.004492,0.000070
98,16,8,total,current,0.009801,0.000087
101,16,14,total,current,0.012204,0.000117
102,16,13,total,current,0.587235,0.000512
131,22,5,total,current,0.000060,0.000009


In [10]:
print(tally['mean'].sum())

1.609714
